# Send WhatsApp Web Messages From Excel With Images

This program send messages via WhatsApp Web with images  
The messages must be stored in an Excel file, and mus contain the following columns:  

CLIENTE: Name of destinatary  
TELEFONE: Phone of destinatary  
MENSAGEM: Message to be sent  

Other columns can be present such as name, address, etc, so, by using Excel text concatenation formulae, to send highly personalized messages, including special characters, icons, emoticons, links, etc.  

With messages, the program send the images selected (jpg, png, or gif).  

Notes:  
 - The program waits a random time betweeen messages to avoid WhatsApp to detect automation.  
 - The program displays a scrolling text, showing the historical of messages with sucess or fail.  
 - The program try to send the message and the images, if there is an error, jumps to the next one.  
 - At the end, the program saves an Excel file with same fields (Cliente, Telefone and Mensagem) and adds a column with sucess (and date of message) or fail.  


Libraries

In [179]:
# import libraries

# Basic Tkinter
import tkinter as tk
from tkinter import filedialog as fd
import tkinter.scrolledtext as st

# PIL to show images on Tkinter
from PIL import Image, ImageTk

# Pandas
import pandas as pd

# Just to get image file name from full path
from pathlib import Path

# Time to allow program wait few seconds during Chrome operations
import time

# To allow randomic waiting times (important to avoid Whatsapp account blocking)
import random

# Datetime to store current date of messages sent
from datetime import date

# necessary libraries for Chrome operations:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys


# pip install webdriver_manager
# This librari updates automatically the Browser Manger (in this case, Chrome)
from webdriver_manager.chrome import ChromeDriverManager

# Necessary to convert messages from ASCII text into URL aceptable addresses (convert special characters, spaces, etc)
import urllib


In [180]:
# global variables

# list of images to send
imgs_path = []

# current image on viewer
i = 0

# stop sending messages
stop_sending_messages = False


Functions

https://stackoverflow.com/questions/74214619/how-to-use-tkinter-after-method-to-delay-a-loop-instead-time-sleep/74215342?noredirect=1#comment131053675_74215342

In [181]:
def tksleep(t):
    # Function to delay process for t seconds
    # emulating time.sleep(seconds)
    # Thanks to link above
    ms = int(t*1000)
    root = tk._get_default_root()
    var = tk.IntVar(root)
    root.after(ms, lambda: var.set(1))
    root.wait_variable(var)

In [182]:
def show_imgs():
    # Event Function to show the i image from list
    
    global tkphoto 
    # Canvas Size
    can_h = 400
    can_w = 400
      
    # Get the element i from list
    photo = Image.open(imgs_path[i])
    
    # Get the picture size (widht, height)
    pic_w, pic_h = photo.size

    # Calculate aspect image ratio
    aspect = pic_w/pic_h

    # if picture is wider than taller, the resizing limit will be the picture widht, limited to canvas width
    if aspect > 1:
        res_w = can_w
        res_h = can_w / aspect
    
    # else, the resizing limit will be the picture height, limited to canvas height
    else:
        res_h = can_h
        res_w = can_h * aspect
    

    # resize picture
    photo = photo.resize((int(res_w),int(res_h)))

    # create the Tkinter picture image object
    tkphoto = ImageTk.PhotoImage(photo) 

    # put the picture into the label
    lbl_photo = tk.Label(image=tkphoto,width=can_w,height=can_h,borderwidth=2,relief='solid')
    lbl_photo.grid(row=1,column=4,rowspan=4, padx=10,pady=10)
    
    # mostrar numero da imagem e o nome
    show_name()
    
    return()

In [183]:
def show_name():
    # Function to show current image number, total number of selected images and current image name (without path)
    # Path.name method extracts the name from a full 
    img_name = '{} de {}: {}'.format(i+1,len(imgs_path),Path(imgs_path[i]).name)
    lbl_imgname = tk.Label(text=img_name,font=('Consolas 10'))
    lbl_imgname.grid(row=5, column=3,columnspan=3,sticky='NSEW',padx=10,pady=10)
    return()
    

In [184]:
def go_rgt():
    # Event function to select next (right) image
    global i
    if i < len(imgs_path)-1:
        i = i + 1
    else:
        i = 0
    
    # Call function to show image number i
    show_imgs()

    return()

In [185]:
def go_lft():
    # Event function to select previous (left) image
    global i
    if i > 0:
        i = i - 1
    else:
        i = len(imgs_path)-1
    
    # Call function to show image number i
    show_imgs()
    
    return()

In [186]:
def sel_imgs():
    # Event function to select image files
        
    # imgs_path is a global list
    global imgs_path

    # i is the number of the image to show
    global i

    images_types = [
            ('Arquivos de imagen','.jpg'),
            ('Arquivos de imagen','.jpeg'),
            ('Arquivos de imagen','.png'),
            ('Arquivos de imagen','.gif'),
            ]

    # Select images
    imgs_path = sorted(list(fd.askopenfilenames(title='Selecione as imagens a enviar',filetypes=images_types)))
    
    # Define show previous image button
    btn_lft = tk.Button(text='<',font=('Consolas 20 bold'),wraplength=100,borderwidth=1,command=go_lft)
    btn_lft.grid(row=1,column=3,rowspan=4,sticky='NSEW',padx=10,pady=10)

    # Define show next image button
    btn_rgt = tk.Button(text='>',font=('Consolas 20 bold'),wraplength=100,borderwidth=1,command=go_rgt)
    btn_rgt.grid(row=1,column=5,rowspan=4,sticky='NSEW',padx=10,pady=10)
        
    # Call function to show first image of selected list
    i = 0
    show_imgs()

    return()

In [187]:
def sel_file():
    # Event function to select Excel file
    
    # contacts_df is the global dataframe with destinataries names, numbers and messages
    global contacts_df

    # file_path is where file is stored
    global file_path

    file_path = fd.askopenfilename(
        title='Selecione o arquivo Excel com a lista de destinatarios',
        filetypes=[('Arquivo Excel','.xls'),('Arquivo Excel','.xlsx')]
        )
    
    # Read Excel file
    contacts_df = pd.read_excel(file_path, sheet_name='CLIENTES')

    # Remove rows with empty messages (this improves process ahead)
    contacts_df = contacts_df[~contacts_df['MENSAGEM'].isnull()]

    # Reset index
    contacts_df.reset_index(inplace=True)

    # Keep just the necessary columns
    contacts_df = contacts_df[['CLIENTE','TELEFONE','MENSAGEM']]
    
    # update informations about number of messages to be sent and
    # inform to click button to start process
    lbl_slctdfile['text'] = 'Serão enviadas {} mensagens.\n Clique em "Enviar Mensagens" para iniciar'.format(contacts_df['MENSAGEM'].count())
    
    return()

Main Process

In [188]:
def wait_wpp_contacts(timetowaith):
    # Function to wait for WhatsApp contacts side bar for x seconds
    # this indicates that the message text input area is ready to receive messages
    while len(msg_browser.find_elements(By.ID,"side")) < 1:
        tksleep(timetowaith)
    return()
    

In [189]:
def stop_sending():
    # Function to stop process
    btn_send.configure(text='')
    global stop_sending_messages
    stop_sending_messages = True
    return()

In [190]:
def send_messages():
    # Send messages process
    # basically, this is a Selenium webscripting process, capturing elements from WhatsApp Web
    
    global msg_browser

    # change button label to Stop and activate stop sending function
    btn_send.configure(text='Stop Process',command=stop_sending)
    
    # Count total number of messages to send
    msg_total = contacts_df['MENSAGEM'].count()

    # Create intance of Google Chorme browsed
    msg_browser = webdriver.Chrome(ChromeDriverManager().install())
    
    # Navigate to WhatsApp Web
    msg_browser.get("https://web.whatsapp.com/")
    # time.sleep(5)
    tksleep(5)

    # Link will open the QR Code authorization
    # Wait until user authorization with cell phone
    
    # Wait to load WhatsApp contacts side bar
    # this indicates it is possible to send messages
    wait_wpp_contacts(2)

    for j, message in enumerate(contacts_df['MENSAGEM']):

        # if stop button was pressed, exit loop
        if stop_sending_messages:
            break

        # this version considers all messages are not null
        # dataframe already cleaned up on opening file function

        # Get customer name and number
        name = contacts_df.loc[j,"CLIENTE"]
        phone = contacts_df.loc[j, "TELEFONE"]
        
        # Update status label
        lbl_sending['text'] = 'Enviando mensagem {} de {}\nPara {} no telefone {}'.format(j+1,msg_total,name,phone)
        mainwindow.update()

        # Convert message from ASCII into URL plain text
        url_message = urllib.parse.quote(f"{message}")

        # build the link
        link = f"https://web.whatsapp.com/send?phone={phone}&text={url_message}"

        # Try to open link; if phone is wrong, it will generate and error, and pass to next
        try:
            # Get link
            msg_browser.get(link)

            # Wait to load WhatsApp contacts side bar
            # this indicates it is possible to send messages
            wait_wpp_contacts(2)
                
            # Send ENTER
            msg_browser.find_element(By.XPATH,'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div/p/span').send_keys(Keys.ENTER)
            
            # time.sleep(3)
            tksleep(3)
            # Wait to load WhatsApp contacts side bar
            # this indicates it is possible to send messages
            # wait_wpp_contacts(1)
            
            # For each image in images list
            for img_file in imgs_path:
        
                # Find attachment button (paperclip) and click on it
                msg_browser.find_element(By.CSS_SELECTOR,"span[data-icon='clip']").click()
                # time.sleep(3)
                tksleep(3)

                # Select to find element and send all keys the img_file as keystrokes
                msg_browser.find_element(By.CSS_SELECTOR,"input[type='file']").send_keys(img_file)
                # time.sleep(6)
                tksleep(6)

                # Click on send attachment (littel triangle)
                msg_browser.find_element(By.CSS_SELECTOR,"span[data-icon='send']").click()
                # time.sleep(3)
                tksleep(3)
            
            # Write on scrolling text box the result of current message sending process
            txt_result = 'Recebeu a mensagem em {}'.format(date.today())
                            
        except:
            # Write on scrolling text box the result of current message sending process
            txt_result = 'NÃO recebeu a mensagem'

        # Print on terminal
        print('{}:{}: {}'.format(j+1,name,txt_result))
            
        # add to result to data frame
        contacts_df.loc[j,'RESULTADO'] = txt_result

        # Write on scrolling text box the result of current message sending process
        txt_sent.insert(tk.INSERT,'{}: {} {}\n'.format(j+1,name,txt_result))
        
        # Point to last line in scrolling text
        txt_sent.see(tk.END)

        # Wait a random time before send next.
        # this is important to avoid WhatsApp to cancel the account due to automation
        tksleep(random.randint(5,10))
    
    # Sending Loop ends here

    lbl_slctdfile['text'] = 'Processo Finalizado'
    lbl_sending['text'] = ''

    # save results dataframe
    result_file = '{}\Resultado Envios {}.xlsx'.format(Path(file_path).parent,date.today())
    contacts_df.to_excel(result_file,index=False)
    
    return()

Main Window Design

In [191]:
# Create application window
mainwindow = tk.Tk()

In [193]:
# Main window title
mainwindow.title("Send messages via WhatsApp")

''

In [194]:
# Main window label title
lbl_title = tk.Label(text="Sent messages via WhatsApp",font=('Consolas 15 bold underline'),borderwidth=1, relief='solid')
lbl_title.grid(row=0, column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [195]:
# Explaining label
lbl_desc = tk.Label(text=
    """This program sent messages through WhatsApp Web,
    together with images, from a list in Excel format.
    The list must contain the following fields or columns:
    Nome, Telefone and Mensagens, in a sheet Clientes
    Each message can be personalized. At the end. stores the
    results in another Excel file"""
    ,font=('Consolas 10'),wraplength=300,borderwidth=1, relief='solid')
lbl_desc.grid(row=1, column=0, columnspan=3,sticky='NSEW',padx=10,pady=10) 

In [196]:
# Excel file selection label
lbl_file = tk.Label(text='Select Excel file with customers, phones and messages:',font=('Consolas 12'),anchor='e')
lbl_file.grid(row=3,column=0,columnspan=2,sticky='NSEW',padx=10,pady=10)

In [197]:
# Excel file selecion button
btn_file = tk.Button(text='Click here to select the file',font=('Consolas 10 bold'),wraplength=100,borderwidth=1,command=sel_file)
btn_file.grid(row=3,column=2,sticky='NSEW',padx=10,pady=10)

In [198]:
# Label with selected Excel file (none at begining, then will show number of message to send)
lbl_slctdfile = tk.Label(text='No file selected',wraplength=500,font=('Consolas 12'),anchor='center')
lbl_slctdfile.grid(row=4,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [199]:
# Image selection label
lbl_imgs = tk.Label(text='Select images to send:',font=('Consolas 12'),anchor='e')
lbl_imgs.grid(row=2,column=0,columnspan=2,sticky='NSEW',padx=10,pady=10)

In [200]:
# Image selection button
btn_imgs = tk.Button(text='Click here to select images',font=('Consolas 10 bold'),wraplength=100,borderwidth=1,command=sel_imgs)
btn_imgs.grid(row=2,column=2,sticky='NSEW',padx=10,pady=10)

In [201]:
# Main process start button
btn_send = tk.Button(text='Send messages',font=('Consolas 10 bold'),command=send_messages)
btn_send.grid(row=5,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [202]:
# Current message information status (who, number, total messages)
lbl_sending = tk.Label(text='',wraplength=500,font=('Consolas 12'),anchor='center')
lbl_sending.grid(row=6,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [203]:
# Scrolling text to show list of sent messages with success or fail
txt_sent = st.ScrolledText(mainwindow,width = 30, 
                            height = 8, 
                            font = ('Consolas 10'))
txt_sent.grid(row=7,column = 0, columnspan=3,sticky='NSEW', pady = 10, padx = 10)

txt_sent.insert(tk.INSERT,'')
# investigate how to make this read only


In [204]:
# define main window icon
mainwindow.iconbitmap(r'icon\whatsapp.ico')

''

In [205]:
# Main window
mainwindow.mainloop()



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
Driver [C:\Users\pgetar\.wdm\drivers\chromedriver\win32\106.0.5249.61\chromedriver.exe] found in cache
C:\Users\pgetar\AppData\Local\Temp/ipykernel_15396/2341606972.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  msg_browser = webdriver.Chrome(ChromeDriverManager().install())


1:Pablo: recebeu a mensagem
2:Pedro with valid message: recebeu a mensagem
3:Pedro Wrong Number: NÃO recebeu a mensagem
4:José valid message: recebeu a mensagem
